# Word Representation

### One Hot Vector를 통한 출력

In [32]:
from konlpy.tag import Mecab

qna_data = [
                ['안녕', '만나서 반가워']
                ,['넌누구니', '나는 AI 봇이란다.']
#                 ,['피자 주문 할께', '페파로니 주문해줘']
#                ,['음료는 멀로', '콜라로 해줘']
            ]
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')

train_data = list(map(lambda x : mecab.morphs(' '.join(x)) , qna_data))

import itertools
train_data = list(itertools.chain.from_iterable(train_data))
#train_data = list(set(train_data))

print(list(train_data))

['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.']


In [33]:
import numpy as np
bucket = np.zeros(len(train_data), dtype=np.float)

for word in train_data :
    bucket_temp = bucket.copy()
    np.put(bucket_temp, train_data.index(word), 1)
    print(bucket_temp)

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


## Word to Vector (By Gensim)
### W2V를 통해 출력해보자

In [34]:
from gensim.models import word2vec

In [35]:
train_data = [train_data]
print(train_data)

model = word2vec.Word2Vec(size=50, window=2, min_count=1)
model.build_vocab(train_data)
model.train(train_data)
print("model check : {0}".format(model))

[['안녕', '만나', '서', '반가워', '넌', '누구', '니', '나', '는', 'AI', '봇', '이', '란다', '.']]
model check : Word2Vec(vocab=14, size=50, alpha=0.025)


In [36]:
import os  
file_path = './model'
if not os.path.exists(file_path):
    os.makedirs(file_path)
model.save(file_path + "/w2v.bin")
model = word2vec.Word2Vec.load("./model/w2v.bin")
print("model load check : {0}".format(model))

model load check : Word2Vec(vocab=14, size=50, alpha=0.025)


In [37]:
print(model.wv.index2word)
X = model[vocab]

['나', '는', '니', '.', '서', '란다', '반가워', '넌', '누구', '봇', '이', 'AI', '안녕', '만나']


NameError: name 'vocab' is not defined

### 안녕, AI등 값의 Vector값 출력 

In [40]:
print(model['안녕'])

[ 0.00759181 -0.0003396   0.00920007 -0.00732685 -0.0073232  -0.00331754
  0.00489599  0.00528371  0.00610048  0.0074512  -0.00556528  0.00851572
 -0.00567806 -0.00255054 -0.00983929  0.00884408 -0.00646101 -0.00799495
 -0.00815    -0.00513075 -0.00993239 -0.00930947 -0.00724646 -0.00952602
  0.00053064 -0.00427199  0.00522723 -0.00521637 -0.00087569  0.00935656
 -0.00459941 -0.00517498  0.00479802  0.00868245  0.00539141 -0.00147727
  0.00581742  0.00830147  0.00336319  0.00915079 -0.00435504 -0.00662863
 -0.00674443 -0.00810368  0.00080349  0.00745528  0.00570942 -0.00193492
 -0.00854987  0.00456542]


In [ ]:
print(model['AI'])

In [ ]:
result1 = model.most_similar(positive='나', negative='', topn=10)
print(result1)

In [ ]:
from sklearn.manifold import TSNE
import pandas as pd
import matplotlib.pyplot as plt


font_name = matplotlib.font_manager.FontProperties(
                fname="/usr/share/fonts/truetype/nanum/NanumGothic.ttf"  # 한글 폰트 위치를 넣어주세요
            ).get_name()
vocab = model.wv.index2word
matplotlib.rc('font', family=font_name)
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X) #t-분포 확률적 임베딩(t-SNE)은 데이터의 차원 축소에 사용
df = pd.concat([pd.DataFrame(X_tsne),
                pd.Series(vocab)],
               axis=1)

df.columns = ['x', 'y', 'word']
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
print(df)
ax.scatter(df['x'], df['y'])
ax.set_xlim(df['x'].max(), df['x'].min())
ax.set_ylim(df['y'].max(), df['y'].min())
for i, txt in enumerate(df['word']):
    ax.annotate(txt, (df['x'].iloc[i], df['y'].iloc[i]))
plt.show()